In [1]:
import numpy as np
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers import Dense, Activation, Dropout, Flatten
from keras import optimizers
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img

Using TensorFlow backend.


In [10]:
IMG_SIZE = 100
LEARNING_RATE = 0.01
EPOCHS = 25
BATCH_SIZE = 32

TRAIN_SIZE = 23000
TEST_SIZE = 2000

TRAIN_DIR = 'data/train'
TEST_DIR = 'data/test'

In [11]:
# datagen = ImageDataGenerator(rescale=1./255)

augmented_gen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

train_gen = augmented_gen.flow_from_directory(
    TRAIN_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    classes=['dogs', 'cats'],
    batch_size=BATCH_SIZE)

test_gen = augmented_gen.flow_from_directory(
    TEST_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    classes=['dogs', 'cats'],
    batch_size=BATCH_SIZE)

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [12]:
model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=(IMG_SIZE, IMG_SIZE, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3), input_shape=(IMG_SIZE, IMG_SIZE, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), input_shape=(IMG_SIZE, IMG_SIZE, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(2))
model.add(Activation('softmax'))

model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 98, 98, 32)        896       
_________________________________________________________________
activation_11 (Activation)   (None, 98, 98, 32)        0         
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 49, 49, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 47, 47, 32)        9248      
_________________________________________________________________
activation_12 (Activation)   (None, 47, 47, 32)        0         
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 23, 23, 32)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 21, 21, 64)        18496     
__________

In [13]:
training = model.fit_generator(
    generator=train_gen,
    steps_per_epoch= TRAIN_SIZE // BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=test_gen,
    validation_steps= TEST_SIZE // BATCH_SIZE)

Epoch 1/25
718/718 [==============================] - 100s 140ms/step - loss: 0.6858 - acc: 0.5446 - val_loss: 0.6558 - val_acc: 0.6290
Epoch 2/25
718/718 [==============================] - 100s 139ms/step - loss: 0.6217 - acc: 0.6575 - val_loss: 0.5752 - val_acc: 0.6759
Epoch 3/25
718/718 [==============================] - 99s 138ms/step - loss: 0.5504 - acc: 0.7254 - val_loss: 0.4692 - val_acc: 0.7868
Epoch 4/25
718/718 [==============================] - 99s 138ms/step - loss: 0.4917 - acc: 0.7689 - val_loss: 0.4330 - val_acc: 0.8065
Epoch 5/25
718/718 [==============================] - 99s 138ms/step - loss: 0.4607 - acc: 0.7906 - val_loss: 0.4140 - val_acc: 0.8049
Epoch 6/25
718/718 [==============================] - 99s 138ms/step - loss: 0.4358 - acc: 0.8048 - val_loss: 0.3866 - val_acc: 0.8327
Epoch 7/25
718/718 [==============================] - 99s 138ms/step - loss: 0.4173 - acc: 0.8171 - val_loss: 0.3755 - val_acc: 0.8327
Epoch 8/25
718/718 [==============================] -

In [17]:
img = load_img('./test/dog.jpg', target_size=(IMG_SIZE, IMG_SIZE))
x = img_to_array(img)
x = x.reshape((1,) + x.shape)
x = x/255

In [15]:
model.save('models/CNN-categorical-25.h5')

In [18]:
prediction = model.predict(x).tolist()
print(prediction[0])

[0.9999943971633911, 5.65083246328868e-06]
